Нам нужен датасет, который содержит информацию о пятистах величайших песен всех времен по версии журнала Rolling Stone по состоянию на 2011 год. Скрапим нужные данные с соответствующей страницы Википедии. 

Код написан с сильной опорой на пример с этого сайта https://pythonru.com/biblioteki/sozdanie-parserov-s-pomoshhju-scrapy-i-python

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import pandas as pd
import re

class WikiSpider(scrapy.Spider):
    name = 'wikispider'
    start_urls = [
        'https://ru.wikipedia.org/wiki/500_величайших_песен_всех_времен_по_версии_журнала_Rolling_Stone']

    def parse(self, response):
        base_url = 'https://ru.wikipedia.org'
        for song in response.css('table tr:not(:first-child)'):
            name = song.css('td:nth-child(2) a::text').get()
            artist = song.css('td:nth-child(3) a::text').getall()
            year = song.css('td:nth-child(4) a::text').get()
            producer = song.css('td:nth-child(6) a::text').getall()
            link = song.css('td:nth-child(2) a::attr(href)').get()
            yield response.follow(url=base_url + link, callback=self.parse_song, cb_kwargs=dict(name=name, artist=artist, year=year, producer=producer))

    def parse_song(self, response, name, artist, year, producer):
        genre_selector = 'th:contains("Жанр") + td.plainlist a::text'
        language_selector = 'th:contains("Язык") + td.plainlist a::text'
        length_selector = 'th:contains("Длительность") + td span::text'
        label_selector = 'th:contains("Лейбл") + td.plainlist a::text'

        genre_raw = response.css(genre_selector).getall()

        # Удалим индексы, которые засоряют наши данные с помощью регулярных выражений
        genres = [re.sub(r'\[[0-9]+\]', '', genre).strip() for genre in genre_raw]
        genres_cleaned = []

        for genre in genres:
            cleaned_genre = re.sub(r',+', ',', genre)
            genres_cleaned.append(cleaned_genre)

        song_2011_data = {
            'name': name,
            'artist': artist,
            'year': year,
            'producer': producer,
            'genre': genres_cleaned,
            'language': response.css(language_selector).getall(),
            'length': response.css(length_selector).get(),
            'label': response.css(label_selector).get(),
        }

        yield song_2011_data

process = CrawlerProcess({
    'FEED_FORMAT': 'csv',
    'FEED_URI': 'song2011).csv'
})
process.crawl(WikiSpider)
process.start()

2024-06-07 21:48:33 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: scrapybot)
2024-06-07 21:48:33 [scrapy.utils.log] INFO: Versions: lxml 5.2.2.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 21.2.0, Python 3.7.8 (tags/v3.7.8:4b47a5b6ba, Jun 28 2020, 08:53:46) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 24.1.0 (OpenSSL 3.2.2 4 Jun 2024), cryptography 42.0.8, Platform Windows-10-10.0.19041-SP0
2024-06-07 21:48:33 [scrapy.crawler] INFO: Overridden settings:
{}
2024-06-07 21:48:33 [py.warnings] WARNING: C:\Users\Lena\AppData\Roaming\Python\Python37\site-packages\scrapy\utils\request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See th